In [ ]:
!pip3 install tinkoff-investments
!pip3 install yfinance
!pip3 install pandas
!pip3 install numpy
!pip3 install decimal
!pip3 install locale

In [48]:
from tinkoff.invest import Client

TOKEN = 't.b7eKSJEp3fpSiiv4mVt4fWwKIxaMHM1lDMtpGsPTeyl850b9Y4MluXYv-EQrj1vEu7QfkNwqGqGPfTW9N6EvTg'

import pandas as pd
from datetime import timedelta
from pathlib import Path
from datetime import datetime, timedelta, timezone
from decimal import Decimal
from tinkoff.invest import CandleInterval, Client, HistoricCandle, Quotation, SubscriptionInterval
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)
from tinkoff.invest.services import MarketDataCache
from tinkoff.invest.utils import now

__all__ = (
    "get_intervals",
    "quotation_to_decimal",
    "decimal_to_quotation",
    "candle_interval_to_subscription_interval",
    "now",
    "candle_interval_to_timedelta",
    "ceil_datetime",
    "floor_datetime",
    "dataclass_from_dict",
    "datetime_range_floor",
)


'''MoneyValue — используется для параметров, у которых есть денежный эквивалент. Возьмем для примера стоимость ценных бумаг — тип состоит из трех параметров:
1) currency — строковый ISO-код валюты, например RUB или USD;
2) units — целая часть суммы;
3) nano — дробная часть суммы, миллиардные доли единицы.
'''
# Quotation type = MoneyValue. We need to convert this to decimal in order to fetch price per share
def quotation_to_decimal(quotation: Quotation) -> Decimal:
    fractional = quotation.nano / Decimal("10e8")
    return Decimal(quotation.units) + fractional

def get_final_stock_volumes(_input):
    return f'Объем: {_input:,} ₽'

def get_final_lots(_lots):
    return f'Кол-во лотов: {_lots:,} шт.'

volumes = []

def main():
    with Client(TOKEN) as client:
        settings = MarketDataCacheSettings(base_cache_dir=Path("market_data_cache"))
        market_data_cache = MarketDataCache(settings=settings, services=client)
        for candle in market_data_cache.get_all_candles(
            figi="BBG004731032",
            from_=now() - timedelta(days=30),
            interval=CandleInterval.CANDLE_INTERVAL_1_MIN,
        ):
            final_stock_volume_rub = int(candle.volume * quotation_to_decimal(candle.close))
            volumes.append(final_stock_volume_rub)
            #print(f'Лукойл (LKOH), Время: {candle.time}, {get_final_lots(candle.volume)}, {get_final_stock_volumes(final_stock_volume_rub)}, Цена: {quotation_to_decimal(candle.close)} ₽')
        
    return 0

if __name__ == "__main__":
    main()
    print(volumes)


[985058, 1915182, 343989, 1640391, 2522778, 311282, 957179, 901421, 1746896, 678316, 613272, 511060, 1602870, 218362, 413449, 116137, 366994, 961722, 385659, 850218, 6128733, 4725490, 3605684, 1022230, 1807294, 5515395, 1412992, 9741613, 4324029, 561984, 232225, 241514, 835830, 111444, 222888, 343619, 1662015, 594240, 2326644, 1091457, 139335, 817520, 3023895, 822165, 585207, 348375, 1454198, 1672740, 808752, 2914296, 1338768, 850767, 739191, 1459786, 1376104, 4113922, 655438, 1045912, 3472429, 4516398, 1221898, 2961094, 2412312, 1677567, 2366086, 1789287, 1189760, 460102, 1278062, 748167, 2040252, 1138637, 1543136, 655368, 976080, 622698, 520464, 906165, 390348, 3095901, 13559674, 11555516, 836010, 32511, 306537, 148624, 106823, 719897, 2142728, 501822, 594944, 20348328, 9705675, 2179944, 1638736, 540038, 498085, 2767940, 1786368, 3366600, 1184475, 5463108, 664592, 97629, 74368, 2086503, 548464, 2073454, 218503, 255722, 832171, 130172, 836820, 111564, 278850, 69720, 762108, 766590, 13

In [23]:
def get_final_stock_volumes(_input):
    return f'Объем: {_input:,} ₽'

def get_final_lots(_lots):
    return f'Кол-во лотов: {_lots:,} шт.'


price = 4666.00000000
lots = 1085
print(f'Цена: {price} ₽')
print(get_final_lots(lots))
output = int(price * lots)
print(output)
print(get_final_stock_volumes(output))


Цена: 4666.0 ₽
Кол-во лотов: 1,085 шт.
5062610
Объем: 5,062,610 ₽


In [91]:
#data = {'Время': ['2023-05-10 13:52:00+00:00', '2023-05-10 13:52:00+00:00'], 'Кол-во лотов': [380, 832852], 'Объем': [2134324, 23455], 'Цена': [4647.00000000, 4645.50000000]}
data = {'Объем': volumes}
df = pd.DataFrame(data)
threshold = 2.0

# Calculate the rolling average and standard deviation of the trading volume
volume_mean = round(df['Объем'].mean())
volume_std = round(df['Объем'].std())

abnormal_volume = (df['Объем'].iloc[-1] - volume_mean) / volume_std

for i in range(len(df)):
    abnormal_volume = (df['Объем'].iloc[i] - volume_mean) / volume_std
    if abnormal_volume > threshold:
        print(f'Зафиксирован аномальный объем: {get_final_stock_volumes(df["Объем"].iloc[i])}')



Зафиксирован аномальный объем: Объем: 61,443,873 ₽
Зафиксирован аномальный объем: Объем: 38,160,307 ₽
Зафиксирован аномальный объем: Объем: 40,805,877 ₽
Зафиксирован аномальный объем: Объем: 70,342,400 ₽
Зафиксирован аномальный объем: Объем: 87,207,310 ₽
Зафиксирован аномальный объем: Объем: 52,929,330 ₽
Зафиксирован аномальный объем: Объем: 32,242,200 ₽
Зафиксирован аномальный объем: Объем: 31,485,140 ₽
Зафиксирован аномальный объем: Объем: 39,591,089 ₽
Зафиксирован аномальный объем: Объем: 41,675,310 ₽
Зафиксирован аномальный объем: Объем: 35,464,575 ₽
Зафиксирован аномальный объем: Объем: 90,950,539 ₽
Зафиксирован аномальный объем: Объем: 137,461,320 ₽
Зафиксирован аномальный объем: Объем: 42,730,039 ₽
Зафиксирован аномальный объем: Объем: 37,933,547 ₽
Зафиксирован аномальный объем: Объем: 47,137,482 ₽
Зафиксирован аномальный объем: Объем: 32,923,867 ₽
Зафиксирован аномальный объем: Объем: 82,073,862 ₽
Зафиксирован аномальный объем: Объем: 63,491,408 ₽
Зафиксирован аномальный объем: